In [6]:
import pandas as pd
# 读取文件
df = pd.read_parquet('/forest/forest/Search-R1-info/data/nq_search/test.parquet')
# 查看前5行数据
print(df.head())
# 查看数据基本信息
# print(df.info())

print("*"*100)

df = pd.read_parquet('/forest/forest/Search-R1-info/data/test_data/test.parquet')
# 查看前5行数据
print(df.head())

       id                                           question  \
0  test_0          who got the first nobel prize in physics?   
1  test_1    when is the next deadpool movie being released?   
2  test_2  which mode is used for short wave broadcast se...   
3  test_3  the south west wind blows across nigeria between?   
4  test_4                what does hp mean in war and order?   

                  golden_answers data_source  \
0       [Wilhelm Conrad Röntgen]          nq   
1                 [May 18, 2018]          nq   
2                 [Olivia, MFSK]          nq   
3               [till September]          nq   
4  [hit points or health points]          nq   

                                              prompt         ability  \
0  [{'content': 'Answer the given question. You m...  fact-reasoning   
1  [{'content': 'Answer the given question. You m...  fact-reasoning   
2  [{'content': 'Answer the given question. You m...  fact-reasoning   
3  [{'content': 'Answer the given ques

In [ ]:
import datasets

# 读取文件
file_path = '/forest/forest/Search-R1-info/data/nq_hotpotqa_train/train.parquet'
ds = datasets.load_dataset('parquet', data_files=file_path)

# 获取数据集（通常是 'train' split）
split_name = list(ds.keys())[0]
df = ds[split_name].to_pandas()

print(f'Total examples: {len(df)}')·
print(f'\nData source distribution:')
print(df['data_source'].value_counts())
print(f'\nHotpotQA count: {len(df[df["data_source"] == "hotpotqa"])}')
print(f'NQ count: {len(df[df["data_source"] == "nq"])}')

/opt/conda/envs/searchr1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 169615 examples [00:01, 95177.73 examples/s]


Total examples: 169615

Data source distribution:
data_source
hotpotqa    90447
nq          79168
Name: count, dtype: int64

HotpotQA count: 90447
NQ count: 79168


In [2]:
# 提取 hotpotqa 数据
hotpotqa_df = df[df['data_source'] == 'hotpotqa'].copy()

print(f'Extracted HotpotQA examples: {len(hotpotqa_df)}')
print(f'\nFirst few rows:')
print(hotpotqa_df.head())

# 保存为 parquet 文件
output_path = '/forest/forest/Search-R1-info/data/my_train_hotpotqa.parquet'
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 使用 datasets 库保存（保持格式一致）
hotpotqa_dataset = datasets.Dataset.from_pandas(hotpotqa_df)
hotpotqa_dataset.to_parquet(output_path)

print(f'\nSaved to: {output_path}')


Extracted HotpotQA examples: 90447

First few rows:
            id                                           question  \
79168  train_0  Which magazine was started first Arthur's Maga...   
79169  train_1  The Oberoi family is part of a hotel company t...   
79170  train_2  Musician and satirist Allie Goertz wrote a son...   
79171  train_3    What nationality was James Henry Miller's wife?   
79172  train_4  Cadmium Chloride is slightly soluble in this c...   

                  golden_answers data_source  \
79168        [Arthur's Magazine]    hotpotqa   
79169                    [Delhi]    hotpotqa   
79170  [President Richard Nixon]    hotpotqa   
79171                 [American]    hotpotqa   
79172                  [alcohol]    hotpotqa   

                                                  prompt         ability  \
79168  [{'content': 'Answer the given question. You m...  fact-reasoning   
79169  [{'content': 'Answer the given question. You m...  fact-reasoning   
79170  [{'conten

Creating parquet from Arrow format: 100%|██████████| 7/7 [00:02<00:00,  2.67ba/s]


Saved to: /forest/forest/Search-R1-info/data/my_train_hotpotqa.parquet


In [8]:
# 统计日志文件中 "info_gain_score" 的出现次数
log_file = '/forest/forest/Search-R1-info/logs/test-11030120-continue-nq-search-r1-grpo-qwen2.5-3b-infogain.log'

# 读取日志文件并统计
with open(log_file, 'r', encoding='utf-8') as f:
    content = f.read()
    count = content.count('info_gain_score')

print(f'File: {log_file}')
print(f'Total occurrences of "info_gain_score": {count}')

# 如果需要查看包含该字符串的行，可以取消下面的注释
# lines_with_keyword = [line for line in content.split('\n') if 'info_gain_score' in line]
# print(f'\nLines containing "info_gain_score": {len(lines_with_keyword)}')
# if lines_with_keyword:
#     print('\nFirst 5 lines:')
#     for i, line in enumerate(lines_with_keyword[:5]):
#         print(f'{i+1}. {line[:200]}...' if len(line) > 200 else f'{i+1}. {line}')

File: /forest/forest/Search-R1-info/logs/test-11030120-continue-nq-search-r1-grpo-qwen2.5-3b-infogain.log
Total occurrences of "info_gain_score": 47616


In [1]:
import wandb
import os
import pandas as pd
import numpy as np
from pathlib import Path
import json
import re

# 设置 wandb 运行目录
wandb_run_dir = '/forest/forest/Search-R1-info/wandb/run-20251109_165031-h8hpdlth'

print("正在读取 wandb 运行数据并提取 val/test_score/* 指标...\n")

# 方法：使用 wandb API 读取历史数据
val_test_scores_by_round = {}  # {round: {data_source: [scores]}}

try:
    # 首先尝试使用 wandb API 读取（如果运行已同步到云端）
    api = wandb.Api()
    
    # 读取元数据以获取可能的 project 信息
    metadata_file = os.path.join(wandb_run_dir, 'files', 'wandb-metadata.json')
    run_id = 'h8hpdlth'
    
    # 尝试常见的 project 名称，或者从目录结构推断
    # 通常 project 名称可以从运行目录的父目录推断
    possible_projects = ['Search-R1', 'Search-R1-open', 'Search-R1-nq_hotpotqa_train']
    
    print("尝试使用 wandb API 读取历史数据...")
    
    run_data = None
    for project in possible_projects:
        try:
            # 尝试读取运行（如果已同步）
            run_path = f"{project}/{run_id}"
            run_data = api.run(run_path)
            print(f"成功从 project '{project}' 读取运行数据")
            break
        except Exception as e:
            continue
    
    if run_data:
        # 读取历史数据
        history = run_data.history()
        print(f"历史数据形状: {history.shape}")
        print(f"历史数据列: {list(history.columns)}")
        
        # 查找所有 val/test_score/* 相关的列
        val_test_score_cols = [col for col in history.columns if 'val/test_score' in col]
        
        if val_test_score_cols:
            print(f"\n找到的 val/test_score 列: {val_test_score_cols}")
            
            # 提取每一轮的数据
            # 使用 _step 列作为轮次标识（如果存在）
            step_col = '_step' if '_step' in history.columns else None
            
            for idx, row in history.iterrows():
                # 使用 step 值作为轮次标识（如果存在），否则使用索引
                if step_col and pd.notna(row[step_col]):
                    round_key = f"step_{int(row[step_col])}"
                else:
                    round_key = f"step_{idx}"
                
                if round_key not in val_test_scores_by_round:
                    val_test_scores_by_round[round_key] = {}
                
                for col in val_test_score_cols:
                    if pd.notna(row[col]):
                        # 从列名中提取 data_source
                        # 格式: val/test_score/{data_source}
                        data_source = col.replace('val/test_score/', '')
                        if data_source not in val_test_scores_by_round[round_key]:
                            val_test_scores_by_round[round_key][data_source] = []
                        val_test_scores_by_round[round_key][data_source].append(float(row[col]))
        else:
            print("历史数据中未找到 val/test_score 列")
            print("可用的指标列:")
            for col in history.columns[:20]:  # 显示前20个列
                print(f"  - {col}")
    else:
        print("运行未同步到云端，将尝试读取本地文件...")
        
        # 尝试读取本地 wandb 文件
        run_file = os.path.join(wandb_run_dir, 'run-mng4jg2z.wandb')
        if os.path.exists(run_file):
            print(f"找到本地运行文件: {run_file}")
            print("注意：本地 .wandb 文件是二进制格式，需要使用 wandb SDK 解析")
            print("建议：使用 'wandb sync' 命令将运行同步到云端，然后使用 API 读取")
    
except Exception as e:
    print(f"使用 wandb API 读取失败: {e}")
    print("将尝试从日志文件读取...")

# 方法2: 使用 wandb 的命令行工具导出数据（如果可用）
print("\n尝试方法2：使用 wandb 命令行工具...")

try:
    import subprocess
    # 尝试使用 wandb 命令行工具导出历史数据
    # wandb sync 可以将离线运行同步，但我们需要的是读取历史
    
    # 更直接的方法：使用 Python 直接解析 wandb 文件
    # 但这比较复杂，因为 .wandb 是二进制格式
    
except Exception as e:
    print(f"方法2失败: {e}")

# 方法3: 使用 wandb 的内部工具直接读取 .wandb 文件
print("\n尝试方法3：直接读取 wandb 历史文件...")

try:
    # 使用 wandb 的内部工具读取历史数据
    from wandb.sdk.internal.file_stream import FileStream
    from wandb.sdk.lib import runid
    
    # wandb 的历史数据可能存储在多个地方
    # 1. run-*.wandb 文件（二进制格式）
    # 2. 如果已同步，可以通过 API 读取
    
    # 尝试使用 wandb 的内部 API 读取
    # 这需要知道如何正确初始化
    
    # 更实用的方法：如果运行已同步，使用 API
    api = wandb.Api()
    
    # 尝试通过运行目录读取（如果支持）
    # 注意：wandb API 通常需要运行已同步到云端
    
    # 如果未同步，我们将依赖日志文件方法
    
except Exception as e:
    print(f"方法3失败: {e}")
    print("将使用日志文件方法...")

# 方法4: 从日志文件中提取 val/test_score 数据（最可靠的方法）
print("\n方法4：从日志文件中提取 val/test_score 数据...")

log_files = list(Path(wandb_run_dir).glob('**/*.log'))
print(f"找到日志文件: {len(log_files)} 个")

# 存储每一轮的 val/test_score 数据
# 格式: {round_number: {data_source: score}}
round_scores = {}

for log_file in log_files:
    try:
        print(f"正在读取: {log_file.name}")
        with open(log_file, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
            
        # 查找包含 val/test_score 的行
        # 可能的格式：
        # - val/test_score/nq: 0.xxx
        # - {"val/test_score/nq": 0.xxx}
        # - val/test_score/nq = 0.xxx
        
        for i, line in enumerate(lines):
            # 匹配各种可能的格式
            patterns = [
                r'val/test_score/(\w+)[:\s=]+([\d.]+)',  # val/test_score/nq: 0.xxx
                r'["\']val/test_score/(\w+)["\']\s*:\s*([\d.]+)',  # "val/test_score/nq": 0.xxx
                r'val/test_score/(\w+)\s+([\d.]+)',  # val/test_score/nq 0.xxx
            ]
            
            for pattern in patterns:
                matches = re.findall(pattern, line)
                for data_source, score_str in matches:
                    try:
                        score = float(score_str)
                        # 尝试推断轮次（从上下文或行号）
                        # 如果日志中有 step 或 epoch 信息，可以提取
                        round_key = None
                        
                        # 查找当前行附近的 step/epoch 信息
                        context_start = max(0, i - 5)
                        context_end = min(len(lines), i + 5)
                        context = ' '.join(lines[context_start:context_end])
                        
                        # 尝试提取 step 或 epoch
                        step_match = re.search(r'step[:\s=]+(\d+)', context, re.IGNORECASE)
                        epoch_match = re.search(r'epoch[:\s=]+(\d+)', context, re.IGNORECASE)
                        global_step_match = re.search(r'global[_\s]?step[:\s=]+(\d+)', context, re.IGNORECASE)
                        
                        if global_step_match:
                            round_key = f"step_{global_step_match.group(1)}"
                        elif step_match:
                            round_key = f"step_{step_match.group(1)}"
                        elif epoch_match:
                            round_key = f"epoch_{epoch_match.group(1)}"
                        else:
                            # 如果没有找到明确的轮次信息，使用行号作为标识
                            round_key = f"round_{i}"
                        
                        if round_key not in round_scores:
                            round_scores[round_key] = {}
                        if data_source not in round_scores[round_key]:
                            round_scores[round_key][data_source] = []
                        round_scores[round_key][data_source].append(score)
                        
                    except ValueError:
                        continue
                        
    except Exception as e:
        print(f"读取日志文件 {log_file} 失败: {e}")

# 合并从 wandb API 和日志文件读取的数据
if val_test_scores_by_round:
    # 如果从 wandb API 读取到了数据，使用它
    round_scores = val_test_scores_by_round
    print("\n使用从 wandb API 读取的数据")
elif not round_scores:
    # 如果都没有数据，尝试使用 round_scores（从日志文件读取的）
    pass

# 整理数据并计算每一轮的平均值
print("\n" + "="*80)
print("每一轮所有 data_source 的平均值:")
print("="*80)

if round_scores:
    round_averages = {}  # {round_key: 该轮所有 data_source 的平均值}
    
    for round_key, scores_dict in sorted(round_scores.items()):
        # 计算每个 data_source 在该轮的平均值
        round_data_source_avgs = []  # 存储该轮每个 data_source 的平均值
        
        for data_source, scores in scores_dict.items():
            avg_score = np.mean(scores)
            round_data_source_avgs.append(avg_score)
        
        # 计算该轮所有 data_source 的平均值
        if round_data_source_avgs:
            round_avg = np.mean(round_data_source_avgs)
            round_averages[round_key] = round_avg
    
    # 显示每一轮的平均值
    if round_averages:
        df_round_avgs = pd.DataFrame([
            {'round': round_key, 'average_score': avg_score}
            for round_key, avg_score in sorted(round_averages.items())
        ])
        print(df_round_avgs.to_string(index=False))
        
        # 计算所有轮次平均值的总体平均
        overall_round_avg = np.mean(list(round_averages.values()))
        print(f"\n所有轮次平均值的总体平均: {overall_round_avg:.4f}")
    else:
        print("未找到有效数据")
        
else:
    print("未找到 val/test_score 数据")
    print("\n提示：")
    print("1. 确保运行已经记录了 val/test_score 指标")
    print("2. 如果运行已同步到 wandb 云端，可以使用 wandb API 读取")
    print("3. 检查日志文件中是否有相关记录")


/opt/conda/envs/searchr1/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/opt/conda/envs/searchr1/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` ty

正在读取 wandb 运行数据并提取 val/test_score/* 指标...

尝试使用 wandb API 读取历史数据...
成功从 project 'Search-R1' 读取运行数据
历史数据形状: (229, 74)
历史数据列: ['timing_s/ref', 'critic/advantages/max', 'state_tokens/coverage', 'critic/rewards/max', 'timing_s/step', 'critic/advantages/mean', 'critic/score/min', '_step', 'env/ratio_of_valid_action', 'reward/output_score_max', 'critic/returns/min', 'reward/output_score_mean', 'critic/score/mean', 'timing_per_token_ms/adv', 'prompt_length/min', 'timing_s/adv', 'actor/kl_coef', 'response_length/mean', 'global_seqlen/balanced_max', 'prompt_length/mean', 'critic/rewards/mean', 'reward/all_score_mean', '_runtime', 'timing_s/update_actor', 'timing_s/gen', 'actor/ppo_kl', 'actor/entropy_loss', 'global_seqlen/minmax_diff', 'timing_per_token_ms/ref', 'actor/pg_clipfrac', 'mfu/actor', 'env/number_of_actions/max', 'env/finish_ratio', 'critic/advantages/min', 'env/number_of_valid_search', 'env/number_of_valid_action', 'reward/info_gain_score_max', 'actor/pg_loss', 'reward/output_score_